In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
class Net (nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), (2, 2))
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [3]:
net = Net()
print(net)

# Net(
#   (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
    # in 1, out 6
#   (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    # in 6, out 16
#   (fc1): Linear(in_features=400, out_features=120, bias=True)
    # in 16 * 5 * 5 = 400, out 120
#   (fc2): Linear(in_features=120, out_features=84, bias=True)
    # in 120, out 84
#   (fc3): Linear(in_features=84, out_features=10, bias=True)
    # in 84, out 10
# )

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [11]:
params = list(net.parameters())
print(len(params))
print(params[0].size())
print(params[1].size())
print(params[2].size())
print(params[3].size())
print(params[4].size())
print(params[5].size())
print(params[6].size())
print(params[7].size())
print(params[8].size())
print(params[9].size())


10
torch.Size([6, 1, 5, 5])
torch.Size([6])
torch.Size([16, 6, 5, 5])
torch.Size([16])
torch.Size([120, 400])
torch.Size([120])
torch.Size([84, 120])
torch.Size([84])
torch.Size([10, 84])
torch.Size([10])


In [12]:
print(params[1])

Parameter containing:
tensor([-0.1295,  0.0074, -0.1438,  0.0280, -0.0489, -0.1349],
       requires_grad=True)


In [14]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[-0.0620, -0.0009,  0.0753, -0.0642,  0.0829,  0.1714, -0.0685, -0.0840,
          0.1040,  0.0039]], grad_fn=<AddmmBackward0>)


In [15]:
net.zero_grad()
out.backward(torch.randn(1, 10))

In [17]:
print(input.unsqueeze(0))

tensor([[[[[-1.4240,  0.4367,  2.2830,  ...,  1.1239, -0.1008,  0.4561],
           [ 0.2804,  0.5691,  2.4466,  ...,  1.6647, -0.8463, -0.4341],
           [ 1.0290,  0.8455, -1.0742,  ...,  0.2011, -0.0384, -1.0703],
           ...,
           [-0.3120, -0.8375, -0.3223,  ..., -1.9305, -0.8917, -0.3415],
           [-0.6836, -0.1955,  1.1044,  ...,  0.0626,  1.8585,  0.7906],
           [ 0.9021,  1.2773,  0.6005,  ..., -0.7168, -0.2447,  0.1748]]]]])


In [18]:
output = net(input)
target = torch.randn(10)
target = target.view(1,-1)
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.8649, grad_fn=<MseLossBackward0>)


In [19]:
print(loss.grad_fn)

In [20]:
print(loss.grad_fn.next_functions[0][0]) # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0]) # ReLU


In [21]:
net.zero_grad()
print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)


conv1.bias.grad before backward
None
conv1.bias.grad after backward
tensor([-0.0070,  0.0104, -0.0041, -0.0122, -0.0107,  0.0007])


In [22]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)
    

In [24]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr = 0.01)

# in your training loop:
optimizer.zero_grad()
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()